In [ ]:
%matplotlib inline

In [ ]:
# import glob, os
# caseid = "HeldSuarez_dt120"
# paths = glob.glob(f"../OUT_3D/{caseid}*.bin3D")
# path = paths[-1][3:]
# path = os.path.splitext(path)[0]

# ncpath = "../" + path + ".nc"
# !docker run -v  $(pwd)/../:/tmp -w /tmp nbren12/sam bin3D2nc {path}.bin3D

# ds = xr.open_dataset(ncpath)

In [ ]:
caseid = "NGAqua_ngaqua.dt15.QOBS"
out_3d_path = "/Users/noah/workspace/models/SAMUWgh/OUT_3D"
out_2d_path = "/Users/noah/workspace/models/SAMUWgh/OUT_2D"
true_path = "/Users/noah/workspace/research/uw-machine-learning/data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/"

In [ ]:
ds = xr.open_mfdataset(f"{out_3d_path}/{caseid}_*.nc", autoclose=True)

# Climate

In [ ]:
mean = ds.mean(['time', 'x'])

In [ ]:
p = mean.p
tm = mean.TABS

tht  = tm * (1000/p)**(2/7)

levels = np.r_[270:350:5]

im = plt.contourf(tht.y, p, tht, levels=levels, extend='both')
plt.colorbar()
plt.clabel(im, colors='black', inline=False, fmt="%.0f")
plt.ylim([1000, 10])
plt.title("Potential Temperature")
plt.xlabel("y")
plt.ylabel("p")
# tht.plot.contourf(levels=levels)

In [ ]:
plt.contourf(tm.y, p, tm)
plt.ylim([1000,10])
plt.colorbar()
plt.title("Absolute Temperature")
plt.xlabel("y")
plt.ylabel("p");

In [ ]:
im = plt.contourf(mean.y, mean.p, mean.U,
                  levels=np.r_[-40:45:5], cmap='RdBu_r',
                  extend='both')


plt.contour(mean.y, mean.p, mean.U, levels=[-10, -8,-6,-4,-2,0], colors='k')
plt.ylim([1000,10])
plt.colorbar(im)
plt.title("Zonal Velocity")
plt.xlabel("y")
plt.ylabel("p");

# Spin-up

Now I plot the temporal evolution of some different fields at a lower atmosphere height as the simulation spins up

Here is the zonal velocity

In [ ]:
ds.U[::20, 5].plot(col='time', col_wrap=4)

and the meridional velocity

In [ ]:
ds.V[::20, 5].plot(col='time', col_wrap=4)

and the absolute temperature

In [ ]:
ds.TABS[::20, 5].plot(col='time', col_wrap=4)

We can see that the equilibrium of the coarse-resolution NG-Aqua is very different than the true simulation. The N-S assymetry is especially strange. Why is that happening? Something with the radiation? We can look at the 2D fields to find this out

In [ ]:
d2d = xr.open_mfdataset(f"../OUT_2D/{caseid}*.nc")

In [ ]:
d2d.SOLIN[5].plot()

mmmm. there is no meridional variation in SOLIN. I must have configured the run incorrectly. Is the diurnal cycle there?

In [ ]:
d2d.SOLIN[:,32, 0].plot()

No. I will have to make sure the radiation matches. How is the SST?

In [ ]:
d2_true = xr.open_dataset(true_path + "2d/all.nc").sortby('time')

In [ ]:
d2_true.SST[0,:,0].plot(label='NG-Aqua')
d2d.SST[0,:,0].plot(label='QOBS')
plt.legend()

That does match either...damn. Maybe I should just initialize it from the netCDF as well? NG-Aqua doesn't look like it's using the QOBS profile weirdly enough. Let's do some reverse engineering.

In [ ]:
y = d2_true.y.values
lat = (y-y.mean()) *2.5e-8 * 2 * np.pi

c = np.cos(lat*1.5)
s = np.sin(lat*1.5)
qobs = 273.15 + 27/2  *( 3 * c**2 - c**4)
# qobs = 273.15 + 27/2 * (2 - s**2 - s**4)

In [ ]:
plt.plot(y, qobs, label='QOBS')
plt.plot(y, d2_true.SST[0,:,0], label='NGAqus')
plt.legend()

I have tried to fix these discrepencies.

In [ ]:
caseid = "NGAqua_ngaqua.dt30.fixRad.fixQOBS"
ds = xr.open_mfdataset(f"{out_2d_path}/{caseid}_*.nc", autoclose=True)

In [ ]:
ds.SST[0,:,0].plot(label='Fixed')
d2_true.SST[0,:,0].plot(label='NG-Aqua')

Nice...the SST patterns line up nicely. What about the insolation?

In [ ]:


y = ds.SOLIN.y
sol_coarse = ds.SOLIN[0,:,0]
sol_true = d2_true.SOLIN[23,:,0]

plt.plot(y, sol_coarse, label='Coarse')
plt.plot(y, sol_true, label='NGAqua')


plt.legend()

In [ ]:
float(ds.SOLIN.max())

In [ ]:
float(d2_true.SOLIN.max())

It looks the the solar constant is not the same between these simulations for some reason. Maybe because I did some averaging in the y-direction?